# Webscraping Bank Product Reviews
BoA Datasource = https://wallethub.com/profile/bank-of-america-13000450i

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import  Options
import pandas as pd
import time
import datetime
from random import uniform, choice

/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
def get_proxies():
    url = "https://www.us-proxy.org//"
    # get the HTTP response and construct soup object
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    proxies = []
    for row in soup.find("table").findAll("tr"):
        tds = row.find_all("td")
        # check allows https and is elite class
        try:
            # print(tds[6], tds[4])
            if tds[6].text == 'yes' and tds[4].text == 'elite proxy':
                ip = tds[0].text.strip()
                port = tds[1].text.strip()
                host = f"{ip}:{port}"
                proxies.append(host)
        except IndexError:
            continue
    return proxies

def select_proxy(proxies):
    proxy = choice(proxies)
    return proxy

In [36]:
def create_driver_session(bank="Wells Fargo",headless=True):
    # Get Free Proxy
    proxies = get_proxies()
    proxy = select_proxy(proxies) # IP:PORT or HOST:PORT
    print('Proxy IP', proxy)
    # Set Proxy address to fetch new data
    options = Options()
    options.add_argument('--proxy-server=%s' % proxy)
    # No need to see page
    options.headless = headless
    # Create driver
    driver = webdriver.Chrome(DRIVER_PATH, options=options)
    # Testing it is functioning URL
    try:
        driver.implicitly_wait(10)
        driver.get(url)
    except Exception as e:
        print('Driver Creation Failed')
        driver.quit()
        create_driver_session(bank=BANK,headless=False)
    if bank not in driver.title:
        print("Did not reach review page - blocked")
        driver.quit()
        create_driver_session(bank=BANK,headless=False)
    else:    
        return driver

In [88]:
def get_html_file(url, driver, page=None):
    if page != None:
        url += "?p=" + page
    # inserting random time lapse to try to prevent bot detection
    random_value = uniform(2.5,7.0)
    print('Waited', random_value, 'seconds')
    time.sleep(random_value) 
    # fetch website
    driver.get(url)
    # get html
    soup = BeautifulSoup(driver.page_source, 'lxml', from_encoding='UTF-8')
    return soup

In [7]:
def fetch_page_reviews(html, df):
    for review in html.find_all(class_='rvtab-citem'):  
        # Initialize Values
        stars = 0
        verified = True
        review_comment = False
        # Name & UserName
        author_name = review.find(class_='rvtab-ci-name').text.strip()
        nickname = review.find(class_='rvtab-ci-nickname').text[1:]
        # Date of Review
        date = review.find('time')['datetime']
        # Product Reviewed
        if review.find(class_='rvtab-ci-category') is not None:
            product = review.find(class_='rvtab-ci-category').text.replace('Product:','').strip()
        else:
            # prints comments that don't belong to specific product, probably comments to previous review
            print(author_name, nickname, date, verified, stars)
            review_comment = True
            product = None
        # Review Comment
        comment = review.find(class_='rvtab-ci-content').text
        # Count number of stars
        for star in review.find_all('path'):
            if star['fill'] == '#4ae0e1':
                stars += 1
        if review.find(class_='rvtab-ci-verified') is None:
            verified = False
            # print(author_name, nickname, date, verified, stars)
            # print(product)
            # print('\t', comment)
        # temporary to add row by row
        row = [[date, author_name, nickname, verified, product, review_comment, stars, comment]]
        temp = pd.DataFrame(row, columns=df.columns)
        # add to main dataframe
        df = df.append(temp, ignore_index=True)
    return df

In [8]:
# TODO:
# 2 potential breaks: 1) if too many requests 2) no windown exists exception
# IF continues to crash try the clicking option instead of a new request

In [89]:
def scrape_reviews(url, driver, start_page=None, data=None):
    if data is None:
        # Create DataFrame for storage
        col_names = ['date', 'name', 'user_id', 'verified', 'product', 'review_comment', 'stars', 'review']
        data = pd.DataFrame(columns=col_names)
    # attempts counter
    attempts = 0
    # fetch starting page
    html = get_html_file(url, driver)
    # get page positions
    page_position = html.find(class_='rvtab-pag-pos').text
    if start_page is None:
        current_page = page_position.split()[0]
    else:
        current_page = str(start_page)
    last_page = page_position.split()[2]
    while int(current_page) <= int(last_page):
        print(current_page)   
        try:
            html = get_html_file(url, driver, current_page)
        except Exception as e:
            # if attempts > 3:
            #     break
            print('Exception trigerred:', e)
            break
        #     driver = create_driver_session(bank=BANK, headless=False)
        #     attempts += 1
        # check comment review exists in page - else bot may be blocked
        if html.find_all(class_='rvtab-citem'):
            # add page reviews
            data = fetch_page_reviews(html, data)
            # get new page number
            current_page = html.find(class_='rvtab-pag-pos').text.split()[0]
            # increase page
            current_page = str(int(current_page) + 1)
        elif html.title.text == 'IP Block':
            print('No more comments retrieved - Bot Blocked')
            # print('Creating driver with new IP address')
            # driver = create_driver_session(bank=BANK, headless=False)
            # if attempts > 3:
            break
            # attempts += 1
    # driver.close()
    return data, current_page

In [79]:
# Enter Bank Name
BANK = "Chase"

In [80]:
banks = {
    "Wells":"wells-fargo-13007950i",
    "America":"bank-of-america-13000450i",
    "Citibank":"citibank-13001291i",
    "Chase":"chase-13001251i"
}
# Select Bank url location
selected_bank = banks[BANK]

In [81]:
# Create Final URL
root_url = "https://wallethub.com/profile/" 
url = root_url + selected_bank
# Use Driver to fetch website
DRIVER_PATH = "/Users/jesidacosta/OneDrive - University of South Florida/ISM6930/group_project/chromedriver"

In [109]:
# Create Headless Session (not GUI)
options = Options()
proxies = get_proxies()
proxy = select_proxy(proxies) # IP:PORT or HOST:PORT
options.add_argument('--proxy-server=%s' % proxy)
options.headless = False
# Create Driver
driver = webdriver.Chrome(DRIVER_PATH, options=options)
print('Proxy IP:', proxy)
driver.get(url + "?p=" + "1")

# # Fetch HTML using Beautiful Soup
# time.sleep(1)
# # soup = BeautifulSoup(driver.page_source, 'lxml', from_encoding='UTF-8')

# # for review in soup.find_all(class_='rvtab-citem'):
# #     author_name = review.find(class_='rvtab-ci-name').text.strip()
# #     date = review.find('time')['datetime']
# #     comment = review.find(class_='rvtab-ci-content').text
# #     print(date, author_name, comment, '\n')

Proxy IP: 47.254.126.34:1088


In [ ]:
# TODO: Need to fix this function
# driver = create_driver_session(bank=BANK,headless=False)
# driver.get(url)

In [87]:
data, page = scrape_reviews(url,driver)
data

Waited 4.098458985500446 seconds


/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


1
Waited 4.0381796080539 seconds
Andron Rabbit andron311 2020-09-25 True 0
ADAM SOMERS adamksomers 2020-10-30 True 0
Clarissa F clari 2021-01-08 True 0
Samuel Arroyo samuel_arroyo26 2021-06-24 True 0
2
Waited 4.162608836925342 seconds
Rocky Grand ullas_idol 2021-09-01 True 0
3
Waited 2.6359723933525516 seconds
4
Waited 4.43229394768546 seconds
5
Waited 2.9036654529412296 seconds
6
Waited 4.275207951382595 seconds
7
Waited 3.1379198943935265 seconds
8
Waited 4.323217218045755 seconds
9
Waited 3.6395043519855736 seconds
10
Waited 2.5014590405078 seconds
11
Waited 2.447055957078476 seconds
12
Waited 2.371189888160149 seconds
13
Waited 1.8136119351914517 seconds
gloriaacorbin gloriaacorbin 2021-06-29 True 0
14
Waited 4.08611521184565 seconds
15
Waited 5.18889141474769 seconds
16
Waited 3.8528916973645138 seconds
17
Waited 2.74675158631494 seconds
samsobkov samsobkov 2021-06-10 True 0
18
Waited 5.472165441881504 seconds
19
Waited 6.134797941621892 seconds
20
Waited 4.256104170197499 seconds

,date,name,user_id,verified,product,review_comment,stars,review
0,2016-09-21,WalletHub,WalletHub,False,Chase Credit Cards,False,3,If you have good or excellent credit and a fam...
1,2019-10-03,Adam McCann,adam_mccann,False,Chase Credit Cards,False,5,Sapphire Preferred is the card of choice for p...
2,2020-09-25,Andron Rabbit,andron311,False,None,True,0,Until they drop your credit limit down to $500...
3,2020-10-30,ADAM SOMERS,adamksomers,False,None,True,0,@andron311 Crazy. I know ..right. I have been ...
4,2021-01-08,Clarissa F,clari,False,None,True,0,@andron311 same thing happened to me right aft...
...,...,...,...,...,...,...,...,...
2045,2014-09-29,Blake Cooper,blake_cooper_3785,False,Chase Credit Cards,False,4,The Amazon.com Credit Card has a sign-up bonus...
2046,2014-09-26,Colton Young,colton.young.77,False,Chase Credit Cards,False,5,"best card I've ever had, it gave me a great in..."
2047,2014-09-26,Brian Stiehler,brian.stiehler.1,False,Chase Credit Cards,False,4,I like the Chase Freedom Visa card because of ...
2048,2014-09-26,Katie Hall,ken_kate09,False,Chase Credit Cards,False,4,The Chase Freedom Visa has several different p...


In [113]:
data, page = scrape_reviews(url,driver,page,data)
data

Waited 4.3366905197252725 seconds


/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


405
Waited 5.589112163907415 seconds


,date,name,user_id,verified,product,review_comment,stars,review
0,2016-09-21,WalletHub,WalletHub,False,Chase Credit Cards,False,3,If you have good or excellent credit and a fam...
1,2019-10-03,Adam McCann,adam_mccann,False,Chase Credit Cards,False,5,Sapphire Preferred is the card of choice for p...
2,2020-09-25,Andron Rabbit,andron311,False,None,True,0,Until they drop your credit limit down to $500...
3,2020-10-30,ADAM SOMERS,adamksomers,False,None,True,0,@andron311 Crazy. I know ..right. I have been ...
4,2021-01-08,Clarissa F,clari,False,None,True,0,@andron311 same thing happened to me right aft...
...,...,...,...,...,...,...,...,...
4210,2011-10-25,Joe Weider,joe_weider,False,Chase Credit Cards,False,2,One would think that airlines would compete ov...
4211,2011-10-21,Joe Weider,joe_weider,False,Chase Credit Cards,False,5,After hearing the news that Chase and United r...
4212,2012-04-14,Won Lee,won.b.lee.7,False,None,True,0,Joe is absolutely correct. One thing i disagr...
4213,2011-10-20,Joe Weider,joe_weider,False,Chase Credit Cards,False,5,I feel bad for anyone who doesn’t know about t...


In [110]:
data, page = scrape_reviews(url,driver,page,data)
data

Waited 6.887109060360675 seconds


/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


365
Waited 3.063122598041941 seconds
366
Waited 2.890867406349318 seconds
367
Waited 3.7946529376696922 seconds
368
Waited 6.654458667112476 seconds
Dustin Gery dustin.gery 2012-07-12 True 0
Megan Mitchell take.me.home.mickey 2012-11-06 True 0
WalletHub WalletHub 2012-07-23 True 0
369
Waited 5.465913718501731 seconds
Bonnie Bird bonnie.bird.5 2012-07-12 True 0
370
Waited 6.7168114232064156 seconds
371
Waited 3.533370459444072 seconds
372
Waited 5.057767876565129 seconds
Won Lee won.b.lee.7 2012-07-09 True 0
373
Waited 2.8530024355793664 seconds
Lizzie Griffith marygriffithwv 2012-09-19 True 0
Meagan Boucher mgnlvsy0o 2012-09-20 True 0
Ryanne Carter ryannechristine.carter 2012-10-09 True 0
WalletHub WalletHub 2013-01-15 True 0
374
Waited 5.843212135998996 seconds
Erin Bjornsson erin.bjornsson 2012-11-30 True 0
375
Waited 4.494398519670519 seconds
376
Waited 3.5052932944499346 seconds
377
Waited 6.317738476356423 seconds
378
Waited 5.7608177921714425 seconds
379
Waited 5.569032047988545 

,date,name,user_id,verified,product,review_comment,stars,review
0,2016-09-21,WalletHub,WalletHub,False,Chase Credit Cards,False,3,If you have good or excellent credit and a fam...
1,2019-10-03,Adam McCann,adam_mccann,False,Chase Credit Cards,False,5,Sapphire Preferred is the card of choice for p...
2,2020-09-25,Andron Rabbit,andron311,False,None,True,0,Until they drop your credit limit down to $500...
3,2020-10-30,ADAM SOMERS,adamksomers,False,None,True,0,@andron311 Crazy. I know ..right. I have been ...
4,2021-01-08,Clarissa F,clari,False,None,True,0,@andron311 same thing happened to me right aft...
...,...,...,...,...,...,...,...,...
4209,2011-10-31,Joe Weider,joe_weider,False,Chase Credit Cards,False,3,It’s getting to be the time of the year when t...
4210,2011-10-25,Joe Weider,joe_weider,False,Chase Credit Cards,False,2,One would think that airlines would compete ov...
4211,2011-10-21,Joe Weider,joe_weider,False,Chase Credit Cards,False,5,After hearing the news that Chase and United r...
4212,2012-04-14,Won Lee,won.b.lee.7,False,None,True,0,Joe is absolutely correct. One thing i disagr...


In [111]:
page

'405'

In [115]:
data.head(20)

,date,name,user_id,verified,product,review_comment,stars,review
0,2016-09-21,WalletHub,WalletHub,False,Chase Credit Cards,False,3,If you have good or excellent credit and a fam...
1,2019-10-03,Adam McCann,adam_mccann,False,Chase Credit Cards,False,5,Sapphire Preferred is the card of choice for p...
2,2020-09-25,Andron Rabbit,andron311,False,None,True,0,Until they drop your credit limit down to $500...
3,2020-10-30,ADAM SOMERS,adamksomers,False,None,True,0,@andron311 Crazy. I know ..right. I have been ...
4,2021-01-08,Clarissa F,clari,False,None,True,0,@andron311 same thing happened to me right aft...
5,2021-06-24,Samuel Arroyo,samuel_arroyo26,False,None,True,0,All Chase knows how to do is screw customers a...
6,2021-09-06,Margaret,margareth_107,True,Chase Credit Cards,False,5,I love that they monitor your card that way no...
7,2021-09-05,Rebekah,chennoa,True,Chase Credit Cards,False,5,Chase has been very good to me and I will use ...
8,2021-09-05,Thelma,thelmag_7,False,Chase Credit Cards,False,5,I really enjoy the convenience of this card. I...
9,2021-09-05,LAURA,releankanaki,True,Chase Credit Cards,False,5,Chase has given me a chance to obtain a Visa S...


In [116]:
# Save Document as Pandas
# DATE = datetime.datetime.now().date()
# data.to_pickle("wellsfargo_wallethub_reviews_{}.pkl".format(DATE))
data.to_json('data/{}_wallethub_reviews.json'.format(BANK.lower()), orient='records')
# data.to_csv("wellsfargo_wallethub_reviews_{}.csv".format(DATE), index=False, encoding='utf-8')

In [117]:
# Encoding seems to be a problem in excel not in 
data.iloc[26,:].review

'CHASE freedom visa card. Be careful this card has a 3% foreign transaction fee. I use my CHASE amazon rewards visa for all foreign transactions.'

In [119]:
df = pd.read_json("data/chase_wallethub_reviews.json", orient='records')
df

,date,name,user_id,verified,product,review_comment,stars,review
0,2016-09-21,WalletHub,WalletHub,False,Chase Credit Cards,False,3,If you have good or excellent credit and a fam...
1,2019-10-03,Adam McCann,adam_mccann,False,Chase Credit Cards,False,5,Sapphire Preferred is the card of choice for p...
2,2020-09-25,Andron Rabbit,andron311,False,None,True,0,Until they drop your credit limit down to $500...
3,2020-10-30,ADAM SOMERS,adamksomers,False,None,True,0,@andron311 Crazy. I know ..right. I have been ...
4,2021-01-08,Clarissa F,clari,False,None,True,0,@andron311 same thing happened to me right aft...
...,...,...,...,...,...,...,...,...
4210,2011-10-25,Joe Weider,joe_weider,False,Chase Credit Cards,False,2,One would think that airlines would compete ov...
4211,2011-10-21,Joe Weider,joe_weider,False,Chase Credit Cards,False,5,After hearing the news that Chase and United r...
4212,2012-04-14,Won Lee,won.b.lee.7,False,None,True,0,Joe is absolutely correct. One thing i disagr...
4213,2011-10-20,Joe Weider,joe_weider,False,Chase Credit Cards,False,5,I feel bad for anyone who doesn’t know about t...


In [45]:
df.iloc[22].comment

'The card has a $39 per year fee. I’ve had it almost 7 years still a low limit - $2000. My other cards have MUCH higher limits. I have great credit.  I keep it to use for internet purchases, it’s easily monitored and keeps my other higher limit cards safe from the web.'

In [162]:
data

,date,name,user_id,verified,product,review_comment,stars,review
0,2021-08-11,WalletHub,WalletHub,False,Wells Fargo Personal Loans,False,4,\nWells Fargo Personal Loan Review\n \nWells ...
1,2020-01-04,aaron saxton,aaron_e_saxton,False,None,True,0,Obviously written by a marketing company. Not ...
2,2021-07-28,Kimberly Askew,kimberlyaskew,False,None,True,0,Agreed company is horrible and shouldn't be al...
3,2021-09-01,Robert,emmyandaj,True,Wells Fargo Credit Cards,False,4,One of my first cards I ever got. They gave m...
4,2021-09-01,Keynan,jordanmathewphillips,True,Wells Fargo Credit Cards,False,5,Thank you for making it easy and wanting to be...
...,...,...,...,...,...,...,...,...
1901,2012-05-29,Dennis Brenes,DBrenes67,False,Wells Fargo Credit Cards,False,4,"As a college student, I use this card a lot, e..."
1902,2012-05-17,natalie,natalia711,False,Wells Fargo Checking,False,3,My first checking account was with Wells Fargo...
1903,2012-05-17,Becky Dillman,becky_dillman_7,False,Wells Fargo Checking,False,4,I have been a long time customer of Wells Farg...
1904,2012-05-15,Trisha,Mariposa77,False,Wells Fargo Checking,False,3,I opened an account with Wells Fargo just over...
